## memo_re 0 init

### to version = "memo_0"
```
초기화 과정입니다.
```

In [27]:
#01,13,21,25,28,31
candidate = {0:1, 1:13, 2:21, 3:25, 4:28, 5:31}
next_round = 1186

In [28]:
## this code was run.
#!pip install keras-tuner
# import numpy as np
# import pandas as pd
import os
from datetime import datetime
from tqdm import tqdm
# print(f'Current Step #01 [now = {datetime.now()}')
from IPython.display import Markdown

from lib.analy_util import ( draw_markdown,
                             draw_graph,
                             draw_scatter,
                             get_pre_bef,
                             draw_average,
                             get_frequency,
                             get_information,
                             analyze_v1,
                             data_prepare,
                             search_model,
                             get_information_v2,
                             get_extdatas,
                             )
print(f'define this code. [{datetime.now()}]')


to_version="memo_0818_5"
next_round = 1186

title = "Metrics"
row_length = 400
new_to_old = True
show_mark_down = True
markdown_table_limit_length = 5

define this code. [2025-08-18 18:48:52.734354]


In [1]:
create_query = f'create table if not exists results_order(id INTEGER PRIMARY KEY AUTOINCREMENT, round short, n1 short, n2 short, n3 short, n4 short, n5 short, n6 short, bonus short);'
!echo "{create_query}" | sqlite3 ../db/metrics.db

In [19]:
def get_results_ordered(round) -> []:
    """ get_candidated """
    sel_query = f'select n0, n1, n2, n3, n4, n5 from results_order where round={round};'
    print('sel_query', sel_query)
    rows = !echo "{sel_query}" | sqlite3 ../db/metrics.db
    if len(rows) > 0:
        cols = rows[0].split("|")
        if len(cols) == 6:
            return [int(i) for i  in cols]
    return []


def update_results_ordered(round: int, nums: [], bonus: int):
    """ update_candidated """
    set_items = []
    item_cnt = 1
    for num in nums:
        set_items.append(f'n{item_cnt}={num}')
        item_cnt += 1
    update_query = f'update results_order set {", ".join(set_items)} bonus={bonus} where round={round};'
    # print('update_query', update_query)
    !echo "{update_query}" | sqlite3 ../db/metrics.db


def insert_results_ordered(round: int, nums: [], bonus: int):
    """insert_candidated"""
    items = []
    for num in nums:
        items.append(f'{num}')
    insert_query = f'insert into results_order(round, n1, n2, n3, n4, n5, n6, bonus) values( {round}, {", ".join(items)}, {bonus});'
    # print('insert_query', insert_query)
    !echo "{insert_query}" | sqlite3 ../db/metrics.db


def insert_metric(round: int, nums: [], bonus: int):
    """ insert_metric """
    result = get_results_ordered(round)
    if len(result) > 0:
        cols = [int(i) for i in result[0].split('|')]
        equal_state = True
        for i in range(len(cols)):
            if cols[i] != nums[i]:
                equal_state = False
                break
        if equal_state == False:
            update_results_ordered(round, nums, bonus)
    else:
        insert_results_ordered(round, nums, bonus)


In [22]:
data113 = """
1139,15 37 12 5 40 30 18
1138,19 20 14 34 16 29 35
1137,33 45 15 4 9 12 26
1136,44 33 42 38 21 35 1
1135,1 21 33 19 13 6 4
1134,19 7 3 9 13 24 23
1133,13 34 29 20 28 14 23
1132,41 34 6 7 28 19 5
1131,27 14 6 2 38 1 33
1130,28 25 19 21 27 15 40
"""

data112 = """
1129,17 28 10 5 34 11 22
1128,33 28 16 5 1 8 45
1127,37 10 15 31 24 30 32
1126,11 9 4 40 37 5 7
1125,33 6 14 40 25 44 30
1124,30 17 34 8 33 3 28
1123,21 34 19 35 24 13 26
1122,6 30 35 34 21 3 22
1121,32 44 38 24 6 31 8
1120,41 38 2 26 31 19 34
"""

data111 = """
1119,13 9 45 12 20 1 3
1118,16 11 14 13 15 45 3
1117,4 30 36 33 3 9 7
1116,17 30 25 16 31 15 32
1115,7 23 12 32 34 36 8
1114,16 33 10 38 19 32 3
1113,20 32 44 13 21 11 8
1112,16 26 20 44 36 42 24
1111,30 13 43 33 3 45 4
1110,11 20 7 22 3 41 24
"""

data110 = """
1109,13 19 12 33 40 10 2
1108,7 39 19 37 26 44 27
1107,30 41 40 6 14 31 29
1106,4 29 1 45 3 42 36
1105,39 16 34 40 37 6 11
1104,38 30 21 7 1 35 2
1103,40 31 12 44 29 10 2
1102,26 13 38 37 22 14 20
1101,7 6 42 36 13 28 41
1100,31 29 26 17 30 43 12
"""

data109 = """
1099,20 3 38 40 43 28 4
1098,24 16 41 43 21 12 15
1097,34 37 40 33 35 14 4
1096,43 1 23 19 16 12 34
1095,40 34 14 8 29 28 12
1094,15 7 6 26 22 40 41
1093,17 43 10 30 35 22 44
1092,45 26 7 19 33 18 37
1091,24 30 23 28 6 20 44
1090,29 40 19 12 21 45 1
"""

run_this = False
if run_this:
    datas = [data109, data110, data111, data112, data113]
else:
    datas = []

for data in datas:
    rows = data.split("\n")
    for row in rows:
        cols = row.split(',')
        if len(cols) == 2:
            round = cols[0]
            nums = [int(i) for i in cols[1].split(' ')]
            bonus = nums[-1]
            insert_metric(round, nums[:-1], bonus)

In [ ]:
histories = datas.split("\n")
for history in histories:
    cols = history.split(',')
    if len(cols) == 8:
        sel_query = f'select * from results_order where round={cols[0]} and n1={cols[1]} and n1={cols[1]} and n2={cols[2]} and n3={cols[3]} and n4={cols[4]} and n5={cols[5]} and n6={cols[6]};'
        results = !echo "{sel_query}" | sqlite3 ../db/metrics.db
        if len(results) == 0:
            insert_query = f'insert into results_order(round, n1, n2, n3, n4, n5, n6, bonus) values({cols[0]}, {cols[1]}, {cols[2]}, {cols[3]}, {cols[4]}, {cols[5]}, {cols[6]}, {cols[7]});'
            !echo "{insert_query}" | sqlite3 ../db/metrics.db

In [33]:
"""
init memo_re
"""

datas = """
1185,17,6,32,28,29,22,38
1184,16,14,23,31,37,25,42
1183,36,4,15,23,17,27,31
1182,31,21,13,25,28,1,22
1181,8,14,20,41,10,33,28
1180,37,40,18,6,12,41,3
1179,44,18,16,24,40,3,21
1178,43,5,27,44,6,11,17
1177,3,43,7,15,19,16,21
1176,30,11,35,9,7,21,29
1175,42,8,4,3,6,32,31
1174,8,17,39,14,36,11,22
1173,30,35,20,1,18,5,3
1172,44,42,40,9,24,7,45
1171,7,3,12,6,17,11,19
1170,3,38,13,28,34,42,25
1169,39,26,24,12,42,5,20
1168,33,9,24,37,21,30,29
1167,35,40,8,31,39,23,24
1166,14,42,25,29,23,27,16
1165,38,29,45,7,6,27,17
1164,25,18,39,23,17,38,22
1163,13,2,33,16,15,43,4
1162,29,28,21,25,22,20,6
1161,2,12,34,24,20,42,37
1160,39,7,36,13,45,18,19
1159,3,28,39,9,38,27,7
1158,38,21,27,25,32,37,20
1157,5,7,20,26,25,12,28
1156,45,39,34,41,30,31,7
1155,19,10,37,27,38,16,13
1154,22,38,32,8,4,26,27
1153,35,9,44,13,1,10,5
1152,31,36,35,37,32,30,5
1151,27,29,9,3,15,2,8
1150,45,18,39,35,9,8,25
1149,21,15,8,32,36,19,38
1148,16,22,13,15,6,3,32
1147,7,37,11,26,24,27,32
1146,6,19,17,11,40,43,28
1145,31,33,2,11,44,37,32
1144,34,12,4,26,3,15,6
1143,36,17,27,16,10,28,6
1142,37,41,30,2,28,8,22
1141,21,26,35,12,7,11,20
1140,38,29,10,22,31,7,15
"""

histories = datas.split("\n")
for history in histories:
    cols = history.split(',')
    if len(cols) == 8:
        sel_query = f'select * from results_order where round={cols[0]} and n1={cols[1]} and n1={cols[1]} and n2={cols[2]} and n3={cols[3]} and n4={cols[4]} and n5={cols[5]} and n6={cols[6]};'
        results = !echo "{sel_query}" | sqlite3 ../db/metrics.db
        if len(results) == 0:
            insert_query = f'insert into results_order(round, n1, n2, n3, n4, n5, n6, bonus) values({cols[0]}, {cols[1]}, {cols[2]}, {cols[3]}, {cols[4]}, {cols[5]}, {cols[6]}, {cols[7]});'
            !echo "{insert_query}" | sqlite3 ../db/metrics.db
